In [11]:
'''
multiprocessing
DataFrame으로 변환하기
excel file 형태로 내보내기
ban 방지 처리
'''

'\nurl 가져올 때 페이징 기능 -> selenium\nmultiprocessing\n기능별 함수화 -> def\n도시별 검색 기능 -> 도/시 번호로(전체 검색 포함)\nDataFrame으로 변환하기\nexcel file 형태로 내보내기\n'

In [12]:
crawling_list = ['물건명', '분양가', '매매가', '소유자', '계약면적', '전용면적', '융자금', '특징', '중개업소',
                 '월관리비', '방수', '욕실수', '입주가능일', '신축일', '보증금', '월세', '중개보수', '공과금', '물건url']

In [13]:
'''
오피스텔 : rletTypeCd=A02
매매 : tradeTypeCd=A1
Highest Year of School Completed : hscpTypeCd=A02
법정동 코드 API http://juso.seoul.go.kr/openapi/helps/SearchApi_jibun.aspx
http://www.code.go.kr/
https://financedata.github.io/posts/korea-area-code.html
'''

'\n오피스텔 : rletTypeCd=A02\n매매 : tradeTypeCd=A1\nHighest Year of School Completed : hscpTypeCd=A02\n법정동 코드 API http://juso.seoul.go.kr/openapi/helps/SearchApi_jibun.aspx\nhttp://www.code.go.kr/\nhttps://financedata.github.io/posts/korea-area-code.html\n'

# Import Libraries

In [14]:
from urllib import request as rq
from bs4 import BeautifulSoup as bs
from datetime import datetime
import pandas as pd
from tqdm import tqdm

# Get province from user

In [112]:
def getProvince(df_village_code):
    city_number = "0.전지역 | \n"
    city_dict = {0: '전지역'}
    city_list = list(df_village_code["도시"].unique())

    for number, city in enumerate(city_list):
        if (number+1) % 3 == 0:
            city_number += str(number+1) + '.' + city + ' | \n'
        else:
            city_number += str(number+1) + '.' + city + ' | '

        city_dict[number+1] = city
        
    while True:
        try:
            province_num = int(input("원하는 도시의 번호를 선택해주세요\n" + city_number))
            province = city_dict[province_num]
        except ValueError as e:
            print("숫자로 입력해주세요.")
            continue
        except KeyError as k:
            print("0 ~ 15 사이의 숫자로 입력해주세요.")
            continue
        break
            
    return province

# Get ```village_code``` 

In [15]:
df_village_code = pd.read_csv('https://goo.gl/tM6r3v', sep='\t', dtype={'법정동코드':str})
df_village_code = df_village_code[df_village_code["폐지여부"] == "존재"]
print(df_village_code.shape)

(20544, 3)


In [16]:
# 동 list
village_lists = list(df_village_code['법정동명'].str.split(' '))
village_list = [x[-1] for x in village_lists]

village_list[:20]

['서울특별시',
 '종로구',
 '청운동',
 '신교동',
 '궁정동',
 '효자동',
 '창성동',
 '통의동',
 '적선동',
 '통인동',
 '누상동',
 '누하동',
 '옥인동',
 '체부동',
 '필운동',
 '내자동',
 '사직동',
 '도렴동',
 '당주동',
 '내수동']

In [17]:
# 도시 list
city_list = [x[0] for x in village_lists]
city_list[:20]

['서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시',
 '서울특별시']

In [18]:
df_village_code["도시"] = city_list
df_village_code["동"] = village_list
df_village_code = df_village_code[df_village_code['동'] != df_village_code['도시']]
df_village_code = df_village_code[~df_village_code['동'].str.endswith('구')]
df_village_code.head()

,법정동코드,법정동명,폐지여부,도시,동
2,1111010100,서울특별시 종로구 청운동,존재,서울특별시,청운동
3,1111010200,서울특별시 종로구 신교동,존재,서울특별시,신교동
4,1111010300,서울특별시 종로구 궁정동,존재,서울특별시,궁정동
5,1111010400,서울특별시 종로구 효자동,존재,서울특별시,효자동
6,1111010500,서울특별시 종로구 창성동,존재,서울특별시,창성동


In [19]:
df_village_code[df_village_code["동"] == "개포동"]['법정동코드'].values[0]

'1168010300'

In [86]:
city_number = "0.전지역 | \n"
city_dict = {0: '전지역'}
city_list = list(df_village_code["도시"].unique())

for number, city in enumerate(city_list):
    if (number+1) % 3 == 0:
        city_number += str(number+1) + '.' + city + ' | \n'
    else:
        city_number += str(number+1) + '.' + city + ' | '
    
    city_dict[number+1] = city
    
print(city_number)
city_dict

0.전지역 | 
1.서울특별시 | 2.부산광역시 | 3.대구광역시 | 
4.인천광역시 | 5.광주광역시 | 6.대전광역시 | 
7.울산광역시 | 8.세종특별자치시 | 9.경기도 | 
10.강원도 | 11.충청북도 | 12.충청남도 | 
13.전라북도 | 14.전라남도 | 15.경상북도 | 
16.경상남도 | 17.제주특별자치도 | 


{0: '전지역',
 1: '서울특별시',
 2: '부산광역시',
 3: '대구광역시',
 4: '인천광역시',
 5: '광주광역시',
 6: '대전광역시',
 7: '울산광역시',
 8: '세종특별자치시',
 9: '경기도',
 10: '강원도',
 11: '충청북도',
 12: '충청남도',
 13: '전라북도',
 14: '전라남도',
 15: '경상북도',
 16: '경상남도',
 17: '제주특별자치도'}

In [24]:
village_list = df_village_code[df_village_code["도시"]=="서울특별시"]["동"]
village_list.head()

2    청운동
3    신교동
4    궁정동
5    효자동
6    창성동
Name: 동, dtype: object

# Get ```contents_urls```

In [43]:
def getContentsUrls(village_list):
    root_url = "https://land.naver.com/"
    searching_url_list = []
    max_page = 20
    
    for village in tqdm(village_list[:20]):
        code = df_village_code[df_village_code["동"] == village]['법정동코드'].values[0]
        for page_number in range(1, max_page+1):
            basic_url = 'https://land.naver.com/article/articleList.nhn?rletTypeCd=A02&tradeTypeCd=A1&hscpTypeCd=A02&cortarNo={code}&articleOrderCode=&siteOrderCode=&cpId=&mapX=&mapY=&mapLevel=&minPrc=&maxPrc=&minWrrnt=&maxWrrnt=&minLease=&maxLease=&minSpc=&maxSpc=&subDist=&mviDate=&hsehCnt=&rltrId=&mnex=&mHscpNo=&mPtpRange=&mnexOrder=&location=1924&ptpNo=&bssYm=&schlCd=&cmplYn=&page={page_number}#_content_list_target'.format(code=code, page_number=page_number)
            basic_html = rq.urlopen(basic_url).read()
            basic_soup = bs(basic_html, "html.parser")
            basic_elems = basic_soup.find_all("span", {"class":"btn_naverlink"})

            if basic_elems == []:
                break

            else:
                for i in range(len(basic_elems)):
                    estate_url = root_url + basic_elems[i].find("a").attrs["href"]
                    searching_url_list.append(estate_url)
                    
    return searching_url_list

In [50]:
def getContentsTitle(searching_soup):
        # title
        try:
            title_elem = searching_soup.find("h2", {"class":"t_adr ellipsis"})
            title = title_elem.text
        except ValueError as e:
            title = "-"
        finally:
            return title

In [51]:
def getContentsPrice(searching_soup):
        # get price
        try:
            rate_basic = searching_soup.find_all("p", {"class":"rate_info"})
            price = rate_basic[1].find_all("span")[0].text
            price = int(price.split("만")[0].replace(",",""))
        except ValueError as e:
            price = "-"
        finally:
            return price

In [54]:
def getContractArea(searching_soup):
    # get contract area
    try:
        rate_basic = searching_soup.find_all("p", {"class":"rate_info"})
        areas = ""
        area = rate_basic[0].find_all("span")
        for i in range(len(area)):
            areas += area[i].text

        areas_list = areas.split('/')
        contract_area = float(areas_list[0])
    except ValueError as e:
            contract_area = "-"
    finally:
        return contract_area

In [55]:
def getExclusiveArea(searching_soup):
    # get exclusive area
    try:
        rate_basic = searching_soup.find_all("p", {"class":"rate_info"})
        areas = ""
        area = rate_basic[0].find_all("span")
        for i in range(len(area)):
            areas += area[i].text

        areas_list = areas.split('/')
        exclusive_area = float(areas_list[1].split('㎡')[0])
    except ValueError as e:
            exclusive_area = "-"
    finally:
        return exclusive_area

In [58]:
def getSpecificFloor(searching_soup):
    # specific floor
    try:
        summary_basic = searching_soup.find("div", {"class":"view_info"}).find_all("div", {"class":"inner"})
        floors = summary_basic[1].text.split("/")
        specific_floor = int(floors[0])
    except ValueError as e:
        specific_floor = "-"
    finally:
        return specific_floor

In [59]:
def getTotalFloor(searching_soup):
    # total floor
    try:
        summary_basic = searching_soup.find("div", {"class":"view_info"}).find_all("div", {"class":"inner"})
        floors = summary_basic[1].text.split("/")
        total_floor = int(floors[1].split("층")[0])
    except ValueError as e:
        total_floor = "-"
    finally:
        return total_floor

In [61]:
def getRooms(searching_soup):
    # rooms
    try:
        summary_basic = searching_soup.find("div", {"class":"view_info"}).find_all("div", {"class":"inner"})
        room_and_bath = summary_basic[3].text.split("/")
        rooms = int(room_and_bath[0])
    except ValueError as e:
        rooms = "-"
    finally:
        return rooms

In [62]:
def getBaths(searching_soup):
    # baths
    try:
        summary_basic = searching_soup.find("div", {"class":"view_info"}).find_all("div", {"class":"inner"})
        room_and_bath = summary_basic[3].text.split("/")
        baths = int(room_and_bath[1].split('개')[0])
    except ValueError as e:
        baths = "-"
    finally:
        return baths

In [63]:
def getLoanAmount(searching_soup):
    # loan amount
    try:
        summary_basic = searching_soup.find("div", {"class":"view_info"}).find_all("div", {"class":"inner"})
        loan_amount = int(summary_basic[5].text.split("만")[0])
    except ValueError as e:
        loan_amount = "-"
    finally:
        return loan_amount

In [64]:
def getMoveable(searching_soup):
    # moveable
    try:
        summary_basic = searching_soup.find("div", {"class":"view_info"}).find_all("div", {"class":"inner"})
        moveable = summary_basic[7].text
    except ValueError as e:
        moveable = "-"
    finally:
        return moveable

In [65]:
def getAdminCost(searching_soup):
    # administration cost
    try:
        summary_basic = searching_soup.find("div", {"class":"view_info"}).find_all("div", {"class":"inner"})
        administration_cost = int(summary_basic[9].text.split("원")[0].strip().replace(",",""))
    except ValueError as e:
        administration_cost = "-"
    finally:
        return administration_cost

In [67]:
def getDepositAmount(searching_soup):
    # deposit amount
    try:  
        summary_basic = searching_soup.find("div", {"class":"view_info"}).find_all("div", {"class":"inner"})
        deposit_and_rentfee = summary_basic[11].text.split("/")  
        deposit = int(deposit_and_rentfee[0].strip())
    except ValueError as e:
        deposit = "-"
    finally:
        return deposit

In [68]:
def getRentFee(searching_soup):
    # rent fee
    try:
        summary_basic = searching_soup.find("div", {"class":"view_info"}).find_all("div", {"class":"inner"})
        deposit_and_rentfee = summary_basic[11].text.split("/")
        rent_fee = int(deposit_and_rentfee[1].split("만")[0].strip())
    except ValueError as e:
        rent_fee = "-"
    finally:
        return rent_fee

In [69]:
def getChar(searching_soup):
    # characteristics
    try:
        summary_basic = searching_soup.find("div", {"class":"view_info"}).find_all("div", {"class":"inner"})
        characteristics = summary_basic[13].text
    except ValueError as e:
        characteristics = "-"
    finally:
        return characteristics

In [70]:
def getInterm(searching_soup):
    # intermediary
    try:
        summary_basic = searching_soup.find("div", {"class":"view_info"}).find_all("div", {"class":"inner"})
        intermediary = summary_basic[15].text
    except ValueError as e:
        intermediary = "-"
    finally:
        return intermediary

In [73]:
def getUtilityBills(searching_soup):
    # utility bills
    try:
        tax_soup = searching_soup.find_all('ul', {"class":"lst_tax"})[1]
        replaceable_chr_list = ['\\', 'n', 't', ',', ' ', '약', '원']
        utility_bills = tax_soup.find("strong", {"class":"highlight"}).text

        for i in replaceable_chr_list:

            utility_bills = utility_bills.replace(i, '').strip()

        utility_bills = int(utility_bills)

    except ValueError as e:
        utility_bills = "-"
    
    finally:
        return utility_bills

In [74]:
def getIntermPay(searching_soup):
    # intermediate pay
    try:
        intermediate_soup = searching_soup.find_all('ul', {"class":"lst_tax"})[0]
        intermediate_pay = int(intermediate_soup.find("strong").text)
    except ValueError as e:
        intermediate_pay = "-"
    finally:
        return intermediate_pay

In [75]:
def getCompletionDate(searching_soup):
    # completion date
    try:
        completion_elem = searching_soup.find("div", {"class":"div_detail"}).find_all("div", {"class":"inner"})
        completion_date = datetime.strptime(completion_elem[-1].text, '%Y.%m.')
    except ValueError as e:
        completion_date = "-"
    finally:
        return completion_date

In [ ]:
def main():
    
    # public village infos
    df_village_code = pd.read_csv('https://goo.gl/tM6r3v', sep='\t', dtype={'법정동코드':str})
    df_village_code = df_village_code[df_village_code["폐지여부"] == "존재"]
    
    # get 동 list
    village_lists = list(df_village_code['법정동명'].str.split(' '))
    village_list = [x[-1] for x in village_lists]
    
    # get 도시 list
    city_list = [x[0] for x in village_lists]
    
    # create village_DataFrame
    df_village_code["도시"] = city_list
    df_village_code["동"] = village_list
    df_village_code = df_village_code[df_village_code['동'] != df_village_code['도시']]
    df_village_code = df_village_code[~df_village_code['동'].str.endswith('구')]
    
    # get final village_list
    village_list = df_village_code["동"]
    
    # get province from user
    province = getProvince(df_village_code)
    
    # filter village_list
    if province == "전지역":
        village_list = village_list
    else:
        village_list = df_village_code[df_village_code["도시"] == province]["동"]
    
    # ----------------
    
    # get contents urls
    searching_url_list = getContentsUrls(village_list)
    
    # create null dict for DataFrame
    
    
    # get informations
    for url in searching_url_list:
        searching_html = rq.urlopen(url).read()
        searching_soup = bs(searching_html, "html.parser")

        title = getContentsTitle(searching_soup)
        price = getContentsPrice(searching_soup)
        contract_area = getContractArea(searching_soup)
        exclusive_area = getExclusiveArea(searching_soup)
        specific_floor = getSpecificFloor(searching_soup)
        total_floor = getTotalFloor(searching_soup)
        rooms = getRooms(searching_soup)
        baths = getBaths(searching_soup)
        loan_amount = getLoanAmount(searching_soup)
        moveable = getMoveable(searching_soup)
        administration_cost = getAdminCost(searching_soup)
        deposit = getDepositAmount(searching_soup)
        rent_fee = getRentFee(searching_soup)
        characteristics = getChar(searching_soup)
        intermediary = getInterm(searching_soup)
        utility_bills = getUtilityBills(searching_soup)
        intermediate_pay = getIntermPay(searching_soup)
        completion_date = getCompletionDate(searching_soup)
        
        

In [ ]:
if __name__ == "__main__":
    main()